# IMPORTS

In [1]:
import sys
sys.path.insert(0, "../..")
import config as cfg
import gc
import os

In [2]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
from metrics import compute_single_col_score, get_tresholds, compute_weird_pred_proba_score
from sklearn.metrics import recall_score
from helper import make_prediction, check_path
from sklearn.model_selection import StratifiedKFold, train_test_split
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [3]:
import optuna
from optuna.integration import CatBoostPruningCallback

In [4]:
import catboost as cb
import catboost.datasets as cbd
import catboost.utils as cbu

# MODEL TRAINING

In [5]:
train = pd.read_pickle(cfg.PREPARED_TRAIN_DATA_PATH)
test = pd.read_pickle(cfg.PREPARED_TEST_DATA_PATH)

In [6]:
X_train, Y_train = train.drop(cfg.TARGETS, axis=1), train[cfg.TARGETS]

In [7]:
pred_proba_oof = pd.DataFrame(data=np.zeros(shape=(len(train), len(cfg.TARGETS))), index=train.index, columns=cfg.TARGETS)
pred_proba_test = pd.DataFrame(data=np.zeros(shape=(len(test), len(cfg.TARGETS))), index=test.index, columns=cfg.TARGETS)
metrics = {}

In [8]:
EXPERIMENT_FAMILY_NAME = 'catboost'
EXPERIMENT_NAME = 'baseline_optuna'
RANDOM_STATE = 77
N_SPLITS = 5
N_RANDOM_SEEDS = 7

In [9]:
CAT_COLS = cfg.CAT_UNORDERED_COLS + cfg.BINARY_COLS

In [10]:
train_x, valid_x, train_y, valid_y = train_test_split(X_train, Y_train, test_size=0.35, random_state=RANDOM_STATE, shuffle=True)

In [11]:
def objective(trial: optuna.Trial) -> float:

    param = {
        "objective": trial.suggest_categorical("objective", ["MultiLogloss", "MultiCrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.5, 10.0, log=False),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        # "grow_policy": trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]),
        "cat_features": CAT_COLS,
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        # "used_ram_limit": "3gb",
        "eval_metric": "MultiLogloss",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    gbm = cb.CatBoostClassifier(**param)

    pruning_callback = CatBoostPruningCallback(trial, "MultiLogloss")
    gbm.fit(
        train_x,
        train_y,
        eval_set=[(valid_x, valid_y)],
        verbose=0,
        early_stopping_rounds=100,
        callbacks=[pruning_callback],
    )

    # evoke pruning manually.
    pruning_callback.check_pruned()

    preds = gbm.predict_proba(valid_x)
    
    score = compute_weird_pred_proba_score(valid_y, preds, sub_std=False)
    return score

In [12]:
study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=25), direction="maximize"
    )
study.optimize(objective, n_trials=500, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-07-05 22:30:30,978] A new study created in memory with name: no-name-9efaf213-19ad-4cdb-875a-47016788860e
/tmp/ipykernel_91218/23460598.py:25: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "MultiLogloss")
[I 2022-07-05 22:30:45,486] Trial 0 finished with value: 0.699271641685009 and parameters: {'objective': 'MultiCrossEntropy', 'colsample_bylevel': 0.0310012036639174, 'l2_leaf_reg': 6.279216967487989, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'subsample': 0.3664815640526435}. Best is trial 0 with value: 0.699271641685009.


[0.7186438879456706, 0.7510429892980228, 0.5926064227035102, 0.6994791666666667, 0.7345857418111754]
0.699271641685009 0.05600027505925528


/tmp/ipykernel_91218/23460598.py:25: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "MultiLogloss")
[I 2022-07-05 22:30:57,441] Trial 1 finished with value: 0.7040503979233936 and parameters: {'objective': 'MultiCrossEntropy', 'colsample_bylevel': 0.05141636576278381, 'l2_leaf_reg': 3.918983203905896, 'depth': 5, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'subsample': 0.2272536675785576}. Best is trial 1 with value: 0.7040503979233936.


[0.7386990662139219, 0.7575730092508617, 0.5776699029126213, 0.7192708333333333, 0.72703917790623]
0.7040503979233936 0.06449399245814065


/tmp/ipykernel_91218/23460598.py:25: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "MultiLogloss")
[I 2022-07-05 22:31:13,805] Trial 2 finished with value: 0.7088401592713843 and parameters: {'objective': 'MultiLogloss', 'colsample_bylevel': 0.040270597113539995, 'l2_leaf_reg': 8.933759613466608, 'depth': 2, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 2 with value: 0.7088401592713843.


...

[0.7288306451612903, 0.7424723381099221, 0.6251555887478217, 0.7140625, 0.7233640191250981]
0.7067770182288264 0.04183487281248427


/tmp/ipykernel_91218/23460598.py:25: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "MultiLogloss")
[I 2022-07-05 22:39:56,189] Trial 16 finished with value: 0.6927351616759173 and parameters: {'objective': 'MultiLogloss', 'colsample_bylevel': 0.07100705192097594, 'l2_leaf_reg': 1.0620414239888065, 'depth': 7, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS'}. Best is trial 12 with value: 0.718427276361126.


[0.7018781833616299, 0.7357609287139488, 0.6045556385362211, 0.7119791666666666, 0.7095018911011204]
0.6927351616759173 0.04552338897666242


/tmp/ipykernel_91218/23460598.py:25: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "MultiLogloss")
[I 2022-07-05 22:40:37,836] Trial 17 finished with value: 0.6979581527955535 and parameters: {'objective': 'MultiLogloss', 'colsample_bylevel': 0.05035480333064966, 'l2_leaf_reg': 5.174948797562265, 'depth': 9, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 5.418775871307282}. Best is trial 12 with value: 0.718427276361126.


[0.7466044142614601, 0.7005260293850898, 0.6007592730893702, 0.7135416666666667, 0.7283593805751802]
0.6979581527955535 0.05096617607002798
Number of finished trials: 18
Best trial:
  Value: 0.718427276361126
  Params: 
    objective: MultiLogloss
    colsample_bylevel: 0.07741827729746069
    l2_leaf_reg: 4.34762224033762
    depth: 8
    boosting_type: Plain
    bootstrap_type: MVS


In [13]:
cv = MultilabelStratifiedKFold(n_splits=N_SPLITS, random_state=RANDOM_STATE, shuffle=True)

# CAT_COLS = cfg.CAT_UNORDERED_COLS

test_pool = cb.Pool(
        data=test,
        cat_features=CAT_COLS)


fold = 0
for train_idx, val_idx in tqdm(cv.split(X_train, Y_train), total=N_SPLITS):

    train_pool = cb.Pool(
        data=X_train.iloc[train_idx], 
        label=Y_train.iloc[train_idx],
        cat_features=CAT_COLS)

    val_pool = cb.Pool(
        data=X_train.iloc[val_idx], 
        label=Y_train.iloc[val_idx],
        cat_features=CAT_COLS)
        
    for random_seed in tqdm(range(N_RANDOM_SEEDS), total=N_RANDOM_SEEDS):

        clf = cb.CatBoostClassifier(
            **trial.params,
            n_estimators=2000,
            random_seed=random_seed,
            early_stopping_rounds=100,
            silent=True
        )

        clf.fit(train_pool, eval_set=val_pool, plot=False)
        
        model_name = f'{EXPERIMENT_NAME}_fold_{fold}_rs_{random_seed}.cbm'
        model_path = os.path.join(cfg.MODELS_PATH, EXPERIMENT_FAMILY_NAME, EXPERIMENT_NAME)
        check_path(model_path)
        clf.save_model(os.path.join(model_path, model_name))
        
        pred_proba_oof.iloc[val_idx, :] += clf.predict_proba(val_pool)
        pred_proba_test.iloc[:, :] += clf.predict_proba(test_pool)
        del clf; gc.collect()
    del train_pool,val_pool; gc.collect() 
        
    fold += 1
pred_proba_oof /= N_RANDOM_SEEDS
pred_proba_test /= (N_SPLITS * N_RANDOM_SEEDS)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

# PREDICT AND SAVE PREDICTIONS

In [14]:
tresholds = get_tresholds(train[cfg.TARGETS], pred_proba_oof)
sample_submission = pd.read_csv(cfg.SAMPLE_SUBMISSION_PATH).set_index('ID')
submission = make_prediction(pred_proba_test, tresholds, sample_submission)

[0.7088062669771051, 0.7201466658507232, 0.6145061150212755, 0.6634733415167849, 0.737991930013127]
0.6889848638758032 0.04463980344631226


In [15]:
## BEST PARAMS
# [0.7088062669771051, 0.7201466658507232, 0.6145061150212755, 0.6634733415167849, 0.737991930013127]
# 0.6889848638758032 0.04463980344631226

In [16]:
submission_path = os.path.join(cfg.SUBMISSION_PATH, EXPERIMENT_FAMILY_NAME)
check_path(submission_path)
submission.to_csv(os.path.join(submission_path, f'{EXPERIMENT_NAME}.csv'))

pred_proba_oof_path = os.path.join(cfg.OOF_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(pred_proba_oof_path)
pred_proba_oof.to_pickle(os.path.join(pred_proba_oof_path, f'{EXPERIMENT_NAME}.pkl'))

pred_proba_test_path = os.path.join(cfg.TEST_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(pred_proba_test_path)
pred_proba_test.to_pickle(os.path.join(pred_proba_test_path, f'{EXPERIMENT_NAME}.pkl'))